## Molecule Generation using Cetane Number (CN)

In [2]:
import os
import sys
import torch
import joblib
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

from torchdrug import core, datasets, models, tasks
from torch import optim

os.environ["CUDA_VISIBLE_DEVICES"] = ""   # hide GPUs from Torch/TorchDrug

# ============================================================
# 1. Project paths & imports from src/
# ============================================================
import joblib
import sys, os

PROJECT_ROOT = "/home/salvina2004/biofuel-ml"
SRC_DIR = os.path.join(PROJECT_ROOT, "src")

sys.path.append(PROJECT_ROOT)
sys.path.append(SRC_DIR)

print("PYTHONPATH updated:", sys.path)
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
# ============================================================
# DEVICE CONFIGURATION
# ============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================
# PROJECT SETUP
# ============================================================
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)

from src.feature_selection import FeatureSelector, prepare_prediction_features
from src.feature_engineering import *

# Load trained model + selector
model = joblib.load(os.path.join(PROJECT_ROOT, "extratrees_model.pkl"))
selector = joblib.load(os.path.join(PROJECT_ROOT, "feature_selector.pkl"))

# ============================================================
# CN PREDICTION
# ============================================================
def predict_cn(smiles: str) -> float:
    X = prepare_prediction_features([smiles], selector)
    return float(model.predict(X)[0])


# ============================================================
# REWARD FUNCTIONS (GPU-SAFE)
# ============================================================
def reward_maximize_cn(smiles_list, device=device):
    rewards = []
    for smi in smiles_list:
        cn = predict_cn(smi)
        rewards.append(min(cn / 120.0, 1.0))
    return torch.tensor(rewards, dtype=torch.float32, device=device)


def reward_target_range_cn(smiles_list, target_min=50, target_max=70, device=device):
    rewards = []
    for smi in smiles_list:
        cn = predict_cn(smi)
        if target_min <= cn <= target_max:
            rewards.append(1.0)
        else:
            d = target_min - cn if cn < target_min else cn - target_max
            rewards.append(float(np.exp(-d / 20.0)))
    return torch.tensor(rewards, dtype=torch.float32, device=device)


def reward_multi_objective(smiles_list, target_cn=60, device=device):
    rewards = []
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            rewards.append(0.0)
            continue

        cn = predict_cn(smi)
        cn_r = max(0, 1 - abs(cn - target_cn) / 100)

        mw = Descriptors.MolWt(mol)
        mw_r = 1.0 if 150 < mw < 300 else max(0, 1 - abs(mw - 225) / 200)

        logp = Descriptors.MolLogP(mol)
        logp_r = 1.0 if 2 < logp < 6 else max(0, 1 - abs(logp - 4) / 5)

        rewards.append(0.7 * cn_r + 0.2 * mw_r + 0.1 * logp_r)

    return torch.tensor(rewards, dtype=torch.float32, device=device)


# ============================================================
# CUSTOM GCPN TASK — REWARD ON GPU
# ============================================================
class CNOptimizationTask(tasks.GCPNGeneration):
    def __init__(self, model, atom_types,
                 optimization_goal="maximize",
                 target_min=50, target_max=70,
                 target_cn=60,
                 **kwargs):

        super().__init__(model, atom_types, task="plogp", **kwargs)

        self.optimization_goal = optimization_goal
        self.target_min = target_min
        self.target_max = target_max
        self.target_cn = target_cn

    def get_reward(self, graph, smiles_list):
        if self.optimization_goal == "maximize":
            return reward_maximize_cn(smiles_list, device=device)

        elif self.optimization_goal == "target_range":
            return reward_target_range_cn(smiles_list,
                                          target_min=self.target_min,
                                          target_max=self.target_max,
                                          device=device)

        elif self.optimization_goal == "multi_objective":
            return reward_multi_objective(smiles_list,
                                          target_cn=self.target_cn,
                                          device=device)

        raise ValueError(f"Unknown reward goal: {self.optimization_goal}")


# ============================================================
# TRAINING PIPELINE (NOW GPU-SAFE)
# ============================================================
def run_cn_optimization(optimization_goal="maximize",
                        target_cn=60,
                        target_min=50, target_max=70,
                        num_epochs=1,
                        batch_size=8,
                        lr=1e-5):

    print("\n==============================")
    print("     CN OPTIMIZATION START    ")
    print("==============================")

    dataset = datasets.ZINC250k("./zinc_data", kekulize=True, atom_feature="symbol")

    gcpn = models.RGCN(
        input_dim=dataset.node_feature_dim,
        num_relation=dataset.num_bond_type,
        hidden_dims=[256, 256, 256, 256],
        batch_norm=False
    ).to(device)

    task = CNOptimizationTask(
        gcpn,
        dataset.atom_types,
        optimization_goal=optimization_goal,
        target_cn=target_cn,
        target_min=target_min,
        target_max=target_max,
        max_edge_unroll=12,
        max_node=38,
        criterion="ppo",
        reward_temperature=1.0,
        agent_update_interval=3,
        gamma=0.9,
    ).to(device)

    optimizer = optim.Adam(task.parameters(), lr=lr)

    solver = core.Engine(
        task,
        dataset,
        None,
        None,
        optimizer,
        gpus=(0,) if torch.cuda.is_available() else None,
        batch_size=batch_size,
        log_interval=10
    )

    print("\n🔥 Starting RL fine-tuning...")
    solver.train(num_epoch=num_epochs)

    solver.save("gcpn_cn_optimized.pkl")
    print("\n💾 Saved: gcpn_cn_optimized.pkl")

    return solver, task


# ============================================================
# MOLECULE GENERATION
# ============================================================
def generate_molecules(task, num_samples=20):
    task.eval()

    with torch.no_grad():
        graphs = task.generate(num_sample=num_samples)

    results = []
    for g in graphs:
        smi = g.to_smiles()
        mol = Chem.MolFromSmiles(smi)
        cn = predict_cn(smi)

        results.append({
            "smiles": smi,
            "cn": cn,
            "mw": Descriptors.MolWt(mol) if mol else None,
            "logp": Descriptors.MolLogP(mol) if mol else None
        })

    return results


def display_results(results, top_k=10):
    df = pd.DataFrame(results)
    df_sorted = df.sort_values("cn", ascending=False)
    print(df_sorted.head(top_k))
    df_sorted.to_csv("generated_molecules_cn.csv", index=False)
    return df_sorted


# ============================================================
# MAIN
# ============================================================
if __name__ == "__main__":

    solver, task = run_cn_optimization(
        optimization_goal="maximize",
        num_epochs=1,
        batch_size=8
    )

    results = generate_molecules(task, num_samples=10)
    display_results(results)


PYTHONPATH updated: ['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/salvina2004/biofuel-ml/torchdrug_env2/lib/python3.10/site-packages', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml/src', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml/src']
Using device: cuda
Connecting to SQLite database...
                                           Fuel_Name  \
1                        (1,1-dimethylethyl)-benzene   
4                           (1-methylpropyl)-benzene   
8                       (2,2-dimethylpropyl)-benzene   
11                      (2-methylpropyl)cyclopentane   
17  (3E,6E)-3,7,11-Trimethyl-1,3,6,10-dodecatetraene   

                       SMILES         cn  
1            CC(C)(C)c1ccccc1   0.000000  
4              CCC(C)c1ccccc1   6.000000  
8        CC(C)(C)CC1=CC=CC=C1   1.136969  
11              CC(C)CC1CCCC1  47.800000  
17  CC(=CCCC(=CCC=C(C)C=C)C)C  26.165620  


100%|██████████| 1233/1233 [00:02<00:00, 416.77it/s]


Feature matrix shape: (1233, 2256)
Valid molecules: 1233
Any NaN in y_cn: False
NaN count in y_cn: 0
Removed: 0 samples
Remaining: 1217 samples
Removed 0 NaN values
New shapes - X: (1217, 2256), y: (1217,)
Total descriptors available: 208


100%|██████████| 1233/1233 [00:02<00:00, 443.25it/s]


Feature matrix shape: (1233, 2256)
Valid molecules: 1233
Any NaN in y_cn: False
NaN count in y_cn: 0
Removed: 0 samples
Remaining: 1217 samples
Removed 0 NaN values
New shapes - X: (1217, 2256), y: (1217,)

     CN OPTIMIZATION START    


...

/home/salvina2004/biofuel-ml/torchdrug_env2/lib/python3.10/site-packages/torchdrug/data/molecule.py:124: UserWarning: Try to apply masks on molecules with stereo bonds. This may produce invalid molecules. To discard stereo information, call `mol.bond_stereo[:] = 0` before applying masks.
  warnings.warn("Try to apply masks on molecules with stereo bonds. This may produce invalid molecules. "


16:20:28   Downloading https://github.com/rdkit/rdkit/raw/master/Contrib/SA_Score/fpscores.pkl.gz to /home/salvina2004/biofuel-ml/torchdrug_env2/lib/python3.10/site-packages/torchdrug/metrics/rdkit/fpscores.pkl.gz
16:20:28   Extracting /home/salvina2004/biofuel-ml/torchdrug_env2/lib/python3.10/site-packages/torchdrug/metrics/rdkit/fpscores.pkl.gz to /home/salvina2004/biofuel-ml/torchdrug_env2/lib/python3.10/site-packages/torchdrug/metrics/rdkit/fpscores.pkl
16:20:29   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:20:29   PPO objective: 1.11305
16:20:29   Penalized logP: -2.19096
16:20:29   Penalized logP (max): -1.09442
16:20:37   1 / 5 molecules are invalid even after 20 resampling


/home/salvina2004/biofuel-ml/torchdrug_env2/lib/python3.10/site-packages/torchdrug/data/molecule.py:124: UserWarning: Try to apply masks on molecules with stereo bonds. This may produce invalid molecules. To discard stereo information, call `mol.bond_stereo[:] = 0` before applying masks.
  warnings.warn("Try to apply masks on molecules with stereo bonds. This may produce invalid molecules. "


16:20:47   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:20:47   PPO objective: 1.00225
16:20:47   Penalized logP: -2.50302
16:20:47   Penalized logP (max): -0.784373
16:20:55   1 / 5 molecules are invalid even after 20 resampling
16:21:02   1 / 8 molecules are invalid even after 20 resampling
16:21:06   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:21:06   PPO objective: 0.819061
16:21:06   Penalized logP: -2.64673
16:21:06   Penalized logP (max): -1.20434
16:21:19   1 / 8 molecules are invalid even after 20 resampling
16:21:24   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:21:24   PPO objective: 0.993255
16:21:24   Penalized logP: -2.58827
16:21:24   Penalized logP (max): 0.375374
16:21:36   1 / 2 molecules are invalid even after 20 resampling
16:21:40   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
16:21:40   PPO objective: 1.00099
16:21:40   Penalized logP: -2.7086
16:21:40   Penalized logP (max): 0.197781
16:21:42   1 / 8 molecules are invalid even after 20 resampling
16:21:44   1 / 8 molecules are invalid even after 20 resamplin

KeyboardInterrupt: 

In [ ]:
import os
import sys
import torch
import joblib
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
# Import Parallel and delayed for CPU parallelization
from joblib import Parallel, delayed 

from torchdrug import core, datasets, models, tasks
from torch import optim

os.environ["CUDA_VISIBLE_DEVICES"] = ""# hide GPUs from Torch/TorchDrug

# ============================================================
# 1. Project paths & imports from src/
# ============================================================

PROJECT_ROOT = "/home/salvina2004/biofuel-ml"
SRC_DIR = os.path.join(PROJECT_ROOT, "src")

sys.path.append(PROJECT_ROOT)
sys.path.append(SRC_DIR)

print("PYTHONPATH updated:", sys.path)
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
# ============================================================
# DEVICE CONFIGURATION
# ============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================
# PROJECT SETUP
# ============================================================
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)

# NOTE: Assuming these imports are correct based on your file structure
from src.feature_selection import FeatureSelector, prepare_prediction_features
from src.feature_engineering import *

# Load trained model + selector
# These are global variables used by the parallel function
model = joblib.load(os.path.join(PROJECT_ROOT, "extratrees_model.pkl"))
selector = joblib.load(os.path.join(PROJECT_ROOT, "feature_selector.pkl"))

# ============================================================
# CN PREDICTION (PARALLEL-FRIENDLY SINGLE FUNCTION)
# ============================================================
def predict_cn_single(smi: str, selector_obj, model_obj) -> float:
    """
    Predicts CN for a single SMILES string using the CPU model.
    This function is designed to be called in parallel by joblib.
    """
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        # Assign a low reward/CN for invalid molecules
        return 0.0 
    
    # Use the provided global selector and model objects
    X = prepare_prediction_features([smi], selector_obj)
    return float(model_obj.predict(X)[0])


# ============================================================
# REWARD FUNCTIONS (PARALLEL CPU-BASED)
# ============================================================
def reward_maximize_cn(smiles_list, device=device):
    """Rewards CN maximization using Joblib parallel execution."""
    
    # Use Joblib to run predict_cn_single across all available CPU cores (n_jobs=-1)
    cn_predictions = Parallel(n_jobs=-1, backend='loky')(
        delayed(predict_cn_single)(smi, selector, model) for smi in smiles_list
    )
    
    # Convert predictions (Python list) to rewards
    rewards = []
    for cn in cn_predictions:
        # Reward is min(CN / 120.0, 1.0)
        rewards.append(min(cn / 120.0, 1.0))
        
    # Transfer the final reward tensor to the GPU in one go
    return torch.tensor(rewards, dtype=torch.float32, device=device)


def reward_target_range_cn(smiles_list, target_min=50, target_max=70, device=device):
    """Rewards CN values within a specific target range using Joblib parallel execution."""
    
    # Use Joblib to run predict_cn_single across all available CPU cores (n_jobs=-1)
    cn_predictions = Parallel(n_jobs=-1, backend='loky')(
        delayed(predict_cn_single)(smi, selector, model) for smi in smiles_list
    )
    
    # Convert predictions (Python list) to rewards
    rewards = []
    for cn in cn_predictions:
        if cn == 0.0 and Chem.MolFromSmiles(smiles_list[cn_predictions.index(cn)]) is None:
            # Skip invalid molecules already handled by predict_cn_single
            rewards.append(0.0)
            continue
            
        if target_min <= cn <= target_max:
            rewards.append(1.0)
        else:
            # Exponential decay reward for values outside the range
            d = target_min - cn if cn < target_min else cn - target_max
            rewards.append(float(np.exp(-d / 20.0)))
            
    # Transfer the final reward tensor to the GPU in one go
    return torch.tensor(rewards, dtype=torch.float32, device=device)


# ============================================================
# CUSTOM GCPN TASK — CN-ONLY REWARD (NO CHANGES NEEDED HERE)
# ============================================================
class CNOptimizationTask(tasks.GCPNGeneration):
    def __init__(self, model, atom_types,
                 optimization_goal="maximize",
                 target_min=50, target_max=70,
                 **kwargs):

        super().__init__(model, atom_types, task="plogp", **kwargs)

        self.optimization_goal = optimization_goal
        self.target_min = target_min
        self.target_max = target_max

    def get_reward(self, graph, smiles_list):
        if self.optimization_goal == "maximize":
            return reward_maximize_cn(smiles_list, device=device)

        elif self.optimization_goal == "target_range":
            return reward_target_range_cn(smiles_list,
                                          target_min=self.target_min,
                                          target_max=self.target_max,
                                          device=device)

        raise ValueError(f"Unknown CN-only reward goal: {self.optimization_goal}. Must be 'maximize' or 'target_range'.")


# ============================================================
# TRAINING PIPELINE (CN-ONLY) (NO CHANGES NEEDED HERE)
# ============================================================
def run_cn_optimization(optimization_goal="maximize",
                        target_min=50, target_max=70,
                        num_epochs=1,
                        batch_size=8,
                        lr=1e-5):

    print("\n==============================")
    print("     CN OPTIMIZATION START    ")
    print("==============================")

    dataset = datasets.ZINC250k("./zinc_data", kekulize=True, atom_feature="symbol")

    gcpn = models.RGCN(
        input_dim=dataset.node_feature_dim,
        num_relation=dataset.num_bond_type,
        hidden_dims=[256, 256, 256, 256],
        batch_norm=False
    ).to(device)

    task = CNOptimizationTask(
        gcpn,
        dataset.atom_types,
        optimization_goal=optimization_goal,
        target_min=target_min,
        target_max=target_max,
        max_edge_unroll=12,
        max_node=38,
        criterion="ppo",
        reward_temperature=1.0,
        agent_update_interval=3,
        gamma=0.9,
    ).to(device)

    optimizer = optim.Adam(task.parameters(), lr=lr)

    solver = core.Engine(
        task,
        dataset,
        None,
        None,
        optimizer,
        gpus=(0,) if torch.cuda.is_available() else None,
        batch_size=batch_size,
        log_interval=10
    )

    print("\n🔥 Starting RL fine-tuning...")
    solver.train(num_epoch=num_epochs)

    solver.save("gcpn_cn_optimized.pkl")
    print("\n💾 Saved: gcpn_cn_optimized.pkl")

    return solver, task


# ============================================================
# MOLECULE GENERATION (Updated to use the new parallel CN function)
# ============================================================
def generate_molecules(task, num_samples=20):
    task.eval()

    with torch.no_grad():
        graphs = task.generate(num_sample=num_samples)

    smiles_list = [g.to_smiles() for g in graphs]
    
    # Use the parallel function to get all CNs at once
    cn_predictions = Parallel(n_jobs=-1, backend='loky')(
        delayed(predict_cn_single)(smi, selector, model) for smi in smiles_list
    )
    
    results = []
    for i, smi in enumerate(smiles_list):
        mol = Chem.MolFromSmiles(smi)
        cn = cn_predictions[i]

        results.append({
            "smiles": smi,
            "cn": cn,
            # MW/LogP are still calculated sequentially here for final display/saving
            "mw": Descriptors.MolWt(mol) if mol else None,
            "logp": Descriptors.MolLogP(mol) if mol else None
        })

    return results


def display_results(results, top_k=10):
    df = pd.DataFrame(results)
    df_sorted = df.sort_values("cn", ascending=False)
    print(df_sorted.head(top_k))
    df_sorted.to_csv("generated_molecules_cn.csv", index=False)
    return df_sorted


# ============================================================
# MAIN
# ============================================================
if __name__ == "__main__":

    solver, task = run_cn_optimization(
        optimization_goal="maximize",
        num_epochs=1,
        # Consider increasing batch_size here to give Joblib more work to parallelize
        batch_size=32 
    )

    results = generate_molecules(task, num_samples=20)
    display_results(results)

PYTHONPATH updated: ['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/salvina2004/biofuel-ml/torchdrug_env2/lib/python3.10/site-packages', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml/src', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml/src', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml/src', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml/src']
Using device: cuda

     CN OPTIMIZATION START    


Loading ./zinc_data/250k_rndm_zinc_drugs_clean_3.csv:  50%|█████     | 249456/498911 [00:01<00:01, 145343.83it/s]
Constructing molecules from SMILES: 100%|██████████| 249455/249455 [03:54<00:00, 1064.11it/s]


17:22:42   Preprocess training set
17:22:42   {'batch_size': 32,
 'class': 'core.Engine',
 'gpus': (0,),
 'gradient_interval': 1,
 'log_interval': 10,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'capturable': False,
               'class': 'optim.Adam',
               'differentiable': False,
               'eps': 1e-08,
               'foreach': None,
               'fused': None,
               'lr': 1e-05,
               'maximize': False,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'agent_update_interval': 3,
          'atom_types': [6, 7, 8, 9, 15, 16, 17, 35, 53],
          'baseline_momentum': 0.9,
          'class': 'tasks.GCPNGeneration',
          'criterion': 'ppo',
          'gamma': 0.9,
          'hidden_dim_mlp': 128,
          'max_edge_unroll': 12,
          'max_node': 38,
          'model': {'activation': 'relu',
                    'batch_norm': False,
            

KeyboardInterrupt: 

In [ ]:
model = model.GIN(input_dim = )